In [ ]:
%matplotlib widget
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import json
import os
import copy

# Fix cut off xlabel
# https://stackoverflow.com/a/17390833
from matplotlib import rcParams
rcParams.update({'figure.autolayout': True})


In [ ]:
def get_list_of_seeds(subchunk_freq_json_path):
    with open(subchunk_freq_json_path) as json_file:
        data = json.load(json_file)
    seeds = [seed for [seed, _counts] in data["per_seed"]]
    return seeds

In [ ]:
def plot_freq_block(data, block_name, filename):
    H = np.zeros((16, 16))
    try:
        H += np.array_split(data[block_name], 16)
    except KeyError:
        pass
    return plot_freq(H, block_name, filename)

def plot_freq_block_sum(data, block_names, filename):
    H = np.zeros((16, 16))
    for block_name in block_names:
        try:
            H += np.array_split(data[block_name], 16)
        except KeyError:
            pass
    block_name = "+".join(block_names)
    return plot_freq(H, block_name, filename)

def plot_freq(H, block_name, filename):    
    print(f"{block_name} mean: {np.mean(H)}, stddev: {np.std(H)}")
    move_to_diamonds = move_to_diamonds_msg(H)

    #plt.imshow(H, interpolation='none')
    #plt.show()
    fig = plt.figure(figsize=(6, 4))

    # https://stackoverflow.com/a/16492880
    ax = fig.add_subplot(111)
    ax.set_title(f'{block_name} frequency\n(subchunk coordinates, centered at diamond ore at (8, 8))\n{move_to_diamonds}', fontdict={'fontsize': 10})
    plt.xlabel('x coordinate')
    plt.ylabel('z coordinate')
    
    # I want to differentiate 0 from low values
    # https://stackoverflow.com/a/22552651
    # mask some 'bad' data, in your case you would have: data == 0
    H = np.ma.masked_where(H == 0, H)
    cmap = copy.copy(matplotlib.cm.get_cmap("viridis"))
    cmap.set_bad(color='black')

    plt.imshow(H, interpolation='none', origin='upper', cmap = cmap)
    ax.set_aspect('equal')
    cax = fig.add_axes([0.12, 0.1, 0.78, 0.7])
    cax.get_xaxis().set_visible(False)
    cax.get_yaxis().set_visible(False)
    cax.patch.set_alpha(0)
    cax.set_frame_on(False)
    plt.colorbar(orientation='vertical')
    #plt.show()
    # Ensure dir exists
    os.makedirs(os.path.dirname(filename), exist_ok=True)
    plt.savefig(filename)
    plt.close()

def sum_of_8_neighbors_and_center(H, idx):
    [i, j] = idx
    min_i = i-1
    min_j = j-1
    max_i = i+2
    max_j = j+2
    # https://stackoverflow.com/a/21396821
    nei = np.take(H, range(min_i,max_i),mode='wrap', axis=0).take(range(min_j,max_j),mode='wrap',axis=1)
    return np.sum(nei)

def sum_of_24_neighbors_and_center(H, idx):
    [i, j] = idx
    min_i = i-2
    min_j = j-2
    max_i = i+3
    max_j = j+3
    # https://stackoverflow.com/a/21396821
    nei = np.take(H, range(min_i,max_i),mode='wrap', axis=0).take(range(min_j,max_j),mode='wrap',axis=1)
    return np.sum(nei)

def item_prob_2_and_sum(data, block_name):
    H = np.zeros((16, 16))
    H += np.array_split(data[block_name], 16)

    z, x = np.where(H == np.amax(H))
    # in case of tie, take the one at index 0
    x = x[0]
    z = z[0]
    # The probability is calculated as sum_of_8_neighbors_of_max_value / sum_of_all_values
    if np.sum(H) == 0:
        prob_1 = 0.0
        prob_2 = 0.0
    else:
        prob_1 = sum_of_8_neighbors_and_center(H, [z, x]) / np.sum(H)
        prob_2 = sum_of_24_neighbors_and_center(H, [z, x]) / np.sum(H)
    return prob_2, np.sum(H)

def move_to_diamonds_msg(H):
    z, x = np.where(H == np.amax(H))
    # in case of tie, take the one at index 0
    x = x[0]
    z = z[0]
    # The probability is calculated as sum_of_8_neighbors_of_max_value / sum_of_all_values
    #prob = np.amax(H) / np.sum(H)
    if np.sum(H) == 0:
        prob_1 = 0.0
        prob_2 = 0.0
    else:
        prob_1 = sum_of_8_neighbors_and_center(H, [z, x]) / np.sum(H)
        prob_2 = sum_of_24_neighbors_and_center(H, [z, x]) / np.sum(H)
    move_to_str = f'To find diamonds move '
    if 8 - x < 0:
        move_to_str += f'{x - 8} blocks west (-x)'
    else:
        move_to_str += f'{8 - x} blocks east (+x)'
    move_to_str += ' and '
    if 8 - z < 0:
        move_to_str += f'{z - 8} blocks north (-z)'
    else:
        move_to_str += f'{8 - z} blocks south (+z)'
    move_to_str += f'.\n{(8-x, 8-z)}, prob_1={prob_1:.03f}, prob_2={prob_2:.03f}'
    return move_to_str

In [ ]:
subchunk_freq_json_path = 'subchunk_freq_centered_at_diamond.json'
seeds = get_list_of_seeds(subchunk_freq_json_path)
print("Data obtained from the average of a 2000x2000 block area from these seeds:")
print(seeds)

with open(subchunk_freq_json_path) as json_file:
    data = json.load(json_file)



In [ ]:
def plot_ores(data, output_dir='plots_centered_at_diamond/ores/'):
    # Plot ores by adding ore + deepslate_ore
    plot_freq_block_sum(data, ['minecraft:coal_ore', 'minecraft:deepslate_coal_ore'], f'{output_dir}/coal_subchunk_freq.png')
    plot_freq_block_sum(data, ['minecraft:copper_ore', 'minecraft:deepslate_copper_ore'], f'{output_dir}/copper_subchunk_freq.png')
    plot_freq_block_sum(data, ['minecraft:diamond_ore', 'minecraft:deepslate_diamond_ore'], f'{output_dir}/diamond_subchunk_freq.png')
    plot_freq_block_sum(data, ['minecraft:emerald_ore', 'minecraft:deepslate_emerald_ore'], f'{output_dir}/emerald_subchunk_freq.png')
    plot_freq_block_sum(data, ['minecraft:gold_ore', 'minecraft:deepslate_gold_ore'], f'{output_dir}/gold_subchunk_freq.png')
    plot_freq_block_sum(data, ['minecraft:iron_ore', 'minecraft:deepslate_iron_ore'], f'{output_dir}/iron_subchunk_freq.png')
    plot_freq_block_sum(data, ['minecraft:lapis_ore', 'minecraft:deepslate_lapis_ore'], f'{output_dir}/lapis_subchunk_freq.png')
    plot_freq_block_sum(data, ['minecraft:redstone_ore', 'minecraft:deepslate_redstone_ore'], f'{output_dir}/redstone_subchunk_freq.png')
    
def plot_interesting_blocks(data, output_dir):
    def plot_per_seed_block(block_name):
        plot_freq_block(x, block_name, f'{output_dir}/{block_name}_subchunk_freq.png')

    for b in [        
        'minecraft:andesite',
        'minecraft:clay',
        'minecraft:deepslate',
        'minecraft:diorite',
        'minecraft:granite',
        'minecraft:infested_deepslate',
        'minecraft:sand',
        'minecraft:spawner',
        'minecraft:stone',
        'minecraft:tuff',
    ]:
        plot_per_seed_block(b)

In [ ]:
plot_ores(data['total'])

In [ ]:
for seed, x in data['per_seed']:
    plot_ores(x, f'plots_centered_at_diamond/seeds/{seed}/ores/')
    plot_interesting_blocks(x, f'plots_centered_at_diamond/seeds/{seed}/')    

In [ ]:
total = data['total']
for block_name, counts in total.items():
    plot_freq_block(total, block_name, f'plots_centered_at_diamond/{block_name}_subchunk_freq.png')

In [ ]:
total = data['total']
scores = []
for block_name, counts in total.items():
    p2, total_count = item_prob_2_and_sum(total, block_name)
    # Also add a nice link
    # https://www.freecodecamp.org/news/how-to-use-html-to-open-link-in-new-tab/
    # TODO: this link may not always work because I do not escape special characters such as "/"
    link_to_plot = f"<a target='_blank' rel='noopener noreferrer' href='https://raw.githubusercontent.com/Badel2/mc_block_stats/master/plots_centered_at_diamond/{block_name}_subchunk_freq.png'>Show plot</a>"
    scores.append([p2, int(total_count), block_name, link_to_plot])

scores.sort(key=lambda x: -x[0])
scores.insert(0, ["prob_2", "total_block_count", "block_name"])

# Display table as html
# https://stackoverflow.com/a/35161699
from IPython.display import HTML, display
html_str = '<table><tr>{}</tr></table>'.format(
       '</tr><tr>'.join(
           '<td>{}</td>'.format('</td><td>'.join(str(_) for _ in row)) for row in scores)
       )
#display(HTML(html_str))

# And write it to a file so it can be committed into the repo
with open('prob_2_centered_at_diamond.html', 'w') as f:
    f.write(html_str)


In [ ]:
subchunk_freq_all = data
subchunk_freq_json_path = 'subchunk_freq_centered_at_diamond_odd_chunks.json'
seeds = get_list_of_seeds(subchunk_freq_json_path)

with open(subchunk_freq_json_path) as json_file:
    data = json.load(json_file)


In [ ]:
def calculate_even_chunks_data_given_all_and_odd_chunks(all_chunks, odd_chunks):
    # structure: {
    # per_seed: [[seed, {block_id: counts}]]
    # total: {block_id: counts}
    # }
    def subtract_counts(all_counts, odd_counts):
        even_counts = copy.deepcopy(all_counts)
        for i, x in enumerate(odd_counts):
            even_counts[i] -= x
        return even_counts

    def seed_idx(data, target_seed):
        i = 0
        for i, (seed, per_seed) in enumerate(data):
            if seed == target_seed:
                return i
        return None
    
    even_chunks = copy.deepcopy(all_chunks)
    
    for block_name, counts in odd_chunks['total'].items():
        even_chunks['total'][block_name] = subtract_counts(all_chunks['total'][block_name], counts)
    
    for i, (seed, per_seed) in enumerate(odd_chunks['per_seed']):
        even_seed_idx = seed_idx(even_chunks['per_seed'], seed)
        #odd_seed_idx = seed_idx(odd_chunks['per_seed'], seed)
        all_seed_idx = seed_idx(all_chunks['per_seed'], seed)
        for block_name, counts in per_seed.items():
            even_chunks['per_seed'][even_seed_idx][1][block_name] = subtract_counts(all_chunks['per_seed'][all_seed_idx][1][block_name], counts)
            
    return even_chunks

data_odd = data
data_even = calculate_even_chunks_data_given_all_and_odd_chunks(subchunk_freq_all, data_odd)

In [ ]:
plot_ores(data_even['total'], output_dir='plots_centered_at_diamond/even_chunks/ores/')
plot_ores(data_odd['total'], output_dir='plots_centered_at_diamond/odd_chunks/ores/')

In [ ]:
for seed, x in data_even['per_seed']:
    plot_ores(x, f'plots_centered_at_diamond/even_chunks/seeds/{seed}/ores/')
    plot_interesting_blocks(x, f'plots_centered_at_diamond/even_chunks/seeds/{seed}/')    
    
for seed, x in data_odd['per_seed']:
    plot_ores(x, f'plots_centered_at_diamond/odd_chunks/seeds/{seed}/ores/')
    plot_interesting_blocks(x, f'plots_centered_at_diamond/odd_chunks/seeds/{seed}/')    